## Przypisanie miesiąca do poszczególnych ofert na podstawie pliku, z którego pochodzą dane

In [ ]:
# dodanie miesiąca, w którym dana oferta sprzedaży była dostępna (wszystkie dane pochodzą z roku 2023)
data_AUG["offerMonth"] = 8
data_SEP["offerMonth"] = 9
data_OCT["offerMonth"] = 10

In [ ]:
data_ALL = pd.concat([data_AUG, data_SEP, data_OCT], ignore_index=True)

In [ ]:
data_ALL.tail()

In [ ]:
data_ALL.count()

In [ ]:
# porównanie duplikatów, z pominięciem dodanej kolumny offerMonth
data_ALL.loc[data_ALL.duplicated(data_ALL.columns.delete(data_ALL.columns.get_loc("offerMonth")))]

In [ ]:
# w poszczególnych miesiącach nie występują duplikaty tej samej oferty, ponieważ metoda duplicated() 
# nie znajduje wierszy jeśli uwzględniona zostanie kolumna "offerMonth"
data_ALL.loc[data_ALL.duplicated()]

# TODO: ta komorka moze byc zbedna po analizie danych

## Usunięcie duplikatów

In [ ]:
data_FILTERED = data_ALL.drop_duplicates(subset=["id"], keep='last')
data_FILTERED

In [ ]:
data_FILTERED.count()

In [ ]:
data_FILTERED = data_FILTERED.reset_index(drop=True)

In [ ]:
data_FILTERED